## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os
import sys

# getting the name of the directory
# where the this file is present.
current = os.path.dirname(os.path.realpath('../World_Weather_Analysis'))
  
# Getting the parent directory name
# where the current directory is present.
parent = os.path.dirname(current)
  
# adding the parent directory to 
# the sys.path.
sys.path.append(parent)

# Import the API key.
from World_Weather_Analysis.config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Victoria,HK,22.2855,114.1577,91.26,77,93,3.00,overcast clouds
1,1,Shingu,JP,33.7333,135.9833,83.34,64,100,2.30,light rain
2,2,The Valley,AI,18.2170,-63.0578,82.26,74,100,20.80,overcast clouds
3,3,Honolulu,US,21.3069,-157.8583,84.02,75,75,21.85,broken clouds
4,4,Imbituba,BR,-28.2400,-48.6703,62.85,83,21,7.00,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Shingu,JP,33.7333,135.9833,83.34,64,100,2.30,light rain
2,2,The Valley,AI,18.2170,-63.0578,82.26,74,100,20.80,overcast clouds
3,3,Honolulu,US,21.3069,-157.8583,84.02,75,75,21.85,broken clouds
8,8,Hami,CN,42.8000,93.4500,83.73,23,47,8.59,scattered clouds
11,11,Padang,ID,-0.9492,100.3543,80.65,71,68,1.63,broken clouds
13,13,Papetoai,PF,-17.4961,-149.8729,81.99,66,37,18.10,scattered clouds
15,15,Sierra Vista,US,31.5545,-110.3037,89.58,32,40,20.71,scattered clouds
20,20,Kahului,US,20.8947,-156.4700,87.82,69,40,25.32,scattered clouds
23,23,Singkang,MY,2.1006,102.7994,82.45,81,98,1.63,overcast clouds
24,24,Georgetown,MY,5.4112,100.3354,82.89,91,20,2.30,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                231
City                   231
Country                231
Lat                    231
Lng                    231
Max Temp               231
Humidity               231
Cloudiness             231
Wind Speed             231
Current Description    231
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Shingu,JP,83.34,light rain,33.7333,135.9833,
2,The Valley,AI,82.26,overcast clouds,18.2170,-63.0578,
3,Honolulu,US,84.02,broken clouds,21.3069,-157.8583,
8,Hami,CN,83.73,scattered clouds,42.8000,93.4500,
11,Padang,ID,80.65,broken clouds,-0.9492,100.3543,
13,Papetoai,PF,81.99,scattered clouds,-17.4961,-149.8729,
15,Sierra Vista,US,89.58,scattered clouds,31.5545,-110.3037,
20,Kahului,US,87.82,scattered clouds,20.8947,-156.4700,
23,Singkang,MY,82.45,overcast clouds,2.1006,102.7994,
24,Georgetown,MY,82.89,few clouds,5.4112,100.3354,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Shingu,JP,83.34,light rain,33.7333,135.9833,Shingu UI Hotel
2,The Valley,AI,82.26,overcast clouds,18.2170,-63.0578,CeBlue Villas & Beach Resort
3,Honolulu,US,84.02,broken clouds,21.3069,-157.8583,Aston at the Executive Centre Hotel
8,Hami,CN,83.73,scattered clouds,42.8000,93.4500,Laoyutang Resort
11,Padang,ID,80.65,broken clouds,-0.9492,100.3543,Mercure Padang
...,...,...,...,...,...,...,...
653,Luwuk,ID,79.65,overcast clouds,-0.9516,122.7875,Hotel Santika Luwuk
656,Veraval,IN,80.47,light rain,20.9000,70.3667,Lords Inn Somnath
660,Nisia Floresta,BR,77.38,few clouds,-6.0911,-35.2086,Rancho Crepusculo
668,Chengde,CN,82.29,broken clouds,40.9725,117.9361,Super 8 Hotel


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))